In [ ]:
import threading
import cv2
import tensorflow as tf
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import os

# Load models
yamnet_model = tf.keras.models.load_model("best_model.keras")  # YAMNet model path
yolo_model = YOLO('yolo_v1 (2).pt')  # YOLO model path

# Ensure required directories exist
os.makedirs("anomalous_videos", exist_ok=True)
os.makedirs("anomalous_audio", exist_ok=True)

# Index-to-label mapping for YAMNet
index_to_label = {
    0: "Normal",
    1: "Threat",
    # Add more labels as per your trained model
}

# === Audio Threat Detection ===
def load_audio(audio_file):
    # Implement audio loading (e.g., using librosa or tf.audio)
    pass  # Replace with actual implementation

def preprocess_audio(audio):
    # Implement audio preprocessing (e.g., resampling, normalization)
    pass  # Replace with actual implementation

def predict_audio(audio_file, confidence_threshold=0.5):
    """
    Predict the class of an audio file using YAMNet.

    Args:
        audio_file (str): Path to the audio file.
        confidence_threshold (float): Minimum confidence for classification.

    Returns:
        str: Predicted class name or 'Unknown'.
        float: Confidence score of the prediction.
    """
    audio = load_audio(audio_file)
    processed_audio = preprocess_audio(audio)
    
    _, yamnet_embeddings, _ = yamnet_model(processed_audio)
    avg_embedding = tf.reduce_mean(yamnet_embeddings, axis=0).numpy().reshape(1, -1)
    
    prediction = yamnet_model.predict(avg_embedding)
    predicted_class_index = np.argmax(prediction, axis=1)[0]
    confidence = prediction[0][predicted_class_index]
    
    if confidence < confidence_threshold:
        return "Unknown", confidence
    
    return index_to_label[predicted_class_index], confidence

# === Video Threat Detection ===
def process_video_stream(video_source=0):
    cap = cv2.VideoCapture(video_source)  # Use 0 for webcam
    if not cap.isOpened():
        print("Error: Could not open video stream")
        return

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS)) or 30

    out = None
    recording = False

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Warning: Failed to capture frame, skipping...")
            continue

        results = yolo_model.predict(frame, conf=0.5, verbose=False)
        annotated_frame = results[0].plot()

        anomalies_detected = False
        if results[0].boxes:
            for box in results[0].boxes:
                cls_id = int(box.cls)
                confidence = box.conf.item()
                label = yolo_model.names[cls_id]

                print(f"[{datetime.now()}] Detected: {label} (Confidence: {confidence:.2f})")
                if label in ["violence", "weaponized"]:
                    anomalies_detected = True

                    if not recording:
                        video_filename = f"anomalous_videos/{label}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.mp4"
                        out = cv2.VideoWriter(video_filename, fourcc, fps, (frame_width, frame_height))
                        recording = True
                        print(f"[{datetime.now()}] Recording started: {video_filename}")
                    
                    if out:
                        out.write(frame)
                    break

        if not anomalies_detected and recording:
            print(f"[{datetime.now()}] Anomaly ended. Stopping recording.")
            recording = False
            if out:
                out.release()
                out = None

        # Optional: Display annotated video stream
        # cv2.imshow("Video Stream", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exit requested. Stopping program.")
            break

    if recording and out:
        out.release()
    cap.release()
    cv2.destroyAllWindows()

# === Unified Main Function ===
def unified_system(audio_file=None, video_source=0):
    # Create threads for video and audio processing
    video_thread = threading.Thread(target=process_video_stream, args=(video_source,))
    audio_thread = None

    if audio_file:
        def audio_task():
            predicted_class, confidence = predict_audio(audio_file, confidence_threshold=0.7)
            if predicted_class == "Unknown":
                print("The audio file does not belong to any trained class.")
            else:
                print(f"Predicted class: {predicted_class}")
                print(f"Confidence: {confidence:.2f}")

        audio_thread = threading.Thread(target=audio_task)

    # Start threads
    video_thread.start()
    if audio_thread:
        audio_thread.start()

    # Wait for threads to finish
    video_thread.join()
    if audio_thread:
        audio_thread.join()

# Entry point
if __name__ == "__main__":
    audio_file_path = "test_audio.mp3"  # Replace with your audio file path
    unified_system(audio_file=audio_file_path, video_source=0)


Exception in thread Thread-6 (audio_task):
Traceback (most recent call last):
  File "c:\Program Files\Python311\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "C:\Users\akhil\AppData\Roaming\Python\Python311\site-packages\ipykernel\ipkernel.py", line 761, in run_closure
    _threading_Thread_run(self)
  File "c:\Program Files\Python311\Lib\threading.py", line 975, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\akhil\AppData\Local\Temp\ipykernel_9928\1789872869.py", line 132, in audio_task
  File "C:\Users\akhil\AppData\Local\Temp\ipykernel_9928\1789872869.py", line 48, in predict_audio
  File "C:\Users\akhil\AppData\Roaming\Python\Python311\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\akhil\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\input_spec.py", line 176, in assert_input_compatibility
    raise ValueError(
ValueErro

[2025-01-10 21:24:27.733321] Detected: non-violence (Confidence: 0.98)
[2025-01-10 21:24:27.834180] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:27.902949] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:27.994531] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.075163] Detected: non-violence (Confidence: 0.96)
[2025-01-10 21:24:28.157632] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.250944] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.335006] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.418380] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.500830] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.565297] Detected: non-violence (Confidence: 0.98)
[2025-01-10 21:24:28.634473] Detected: non-violence (Confidence: 0.97)
[2025-01-10 21:24:28.699314] Detected: non-violence (Confidence: 0.98)
[2025-01-10 21:24:28.790273] Detected: non-violence (Confidence: 0.98)
[2025-